In [1]:
## LIBRARIES

# Import libraries
import pandas as pd
import numpy as np
import openai
from openai import OpenAI
from IPython.display import display, Markdown
import os

# Define absolute python path
import sys
sys.path.insert(0, '/Users/giorgiobolchi2/Documents/JRC/LLM') 



## FUNCTIONS

# Load API and import request function
from Code.API import get_chat_response



## DATA

# Load NACE data
from Data.NACEdata import NACElevel0, NACElevel1, NACElevel2, NACElevel3     

# Import all target data (target_code + target_content)
targets_pd = pd.read_csv('/Users/giorgiobolchi2/Documents/JRC/LLM/Data/XLSX_target_data_v1.2_LLM.csv', sep=";")



In [2]:
# GENERATE ANSWERS


# Select data to loop through


# TA = (['TA1','TA1','TA1',   # n=3xTA
#        'TA2','TA2','TA2',
#        'TA3','TA3','TA3',
#        'TA4','TA4','TA4',
#        'TA5','TA5','TA5',
#        'TA6','TA6','TA6',
#        'TA7','TA7','TA7',]) 
#TA = ['TA1','TA1','TA1'] # if want to test on only TA (smaller subset to go faster)
#TA = ['TA1','TA2','TA3','TA4','TA5','TA6','TA7'] # n= 1xTA
TA = ['TA6'] # n=1xTA1

# Model parameters
seed = None 
temperature = 0.2
model = "gpt-4o" 


date= '0207' # to indicate date in filenames
output_directory = f'/Users/giorgiobolchi2/Documents/JRC/LLM/Data/Outputs/{date}/'


# (Loop tools)
loop_counter = 0
answers_content = []
answers_metadata = pd.DataFrame(columns=["TA",
                                         "replicate"
                                         "seed",
                                         "temperature",
                                         "system_fingerprint", 
                                         "prompt_tokens", 
                                         "completion_tokens"])  # create empty panda dataframe with the following columns so to gather a bit more data on the responses and ultimately try to assess consistency

# Loop
for x in range(len(TA)):

    # Subset data to avoid overloading the GPT
    target_subset = targets_pd[targets_pd['target_code'].str.contains(TA[x])]  # subset rows containing one of the characters in  TA[] (ie, select only a specific TA and its targets, because selecting everything in one go is too big for the AI to process)                                                                                                                                          # eg: TA[0] = 'TA1'
    target_list = [f"{row['target_code']}: {row['target_content']}" for index, row in target_subset.iterrows()] # Concatenate target_code and target_content into a list so that it can be added to the prompt as text
    
    
    # Define request
    prompt = f'''Hello,

            Data input: please get acquainted with the following data:
            - NACE classification categories:  {NACElevel1} + {NACElevel2} + {NACElevel3}.
            - List of targets: {target_list}.
            
            Task: 
            - For each target, analyse its content description and assign to each target a NACE category for each level (1,2,3). 

            Answer format: provide your answer as a table in csv format please (separator: ";"), with the following columns:
            - target_code (e.g., TA1.9)
            - target_content (e.g., The contribution of the sectors covered by the EU ETS with respect to the EU Climate ambition should be of -62 % compared to 2005 (increasing the linear emissions reduction factor from 2.2 % per year up to 4.4 %)) 
            - NACE_level1 (e.g., D - Electricity, Gas, Steam and Air Conditioning Supply)
            - NACE_level1_extra1 (e.g.if other categories overlap)
            - NACE_level1_extra2 (e.g.if other categories overlap)
            - NACE_level2 (e.g., D35 - Electricity, gas, steam and air conditioning supply)
            - NACE_level2_extra1 (e.g.if other categories overlap)
            - NACE_level2_extra2 (e.g.if other categories overlap)
            - NACE_level_3 (e.g., D35.1 - Electric power generation, transmission and distribution)
            - NACE_level_3_extra1 (e.g.if other categories overlap)
            - NACE level3_extra2 (e.g.if other categories overlap)
            - justification
            - confidence_score (e.g. confidence value from 0 to 10 about the assignation choices that are made)

            Specifications:
            - If there is some overlap, add the multiple possible fitting NACE categories (up to maximum 3 per NACE level). 
            - Include the name of the NACE categories.
            - Don't forget to provide the title of the target. 
            - If there is no target content, do not invent new content, just state it as empty.
            - For each target, write one to two sentences justifying your choice.
            - Output only the csv table and no additional commentary text.

            Thank you.'''

    # Generate answer
    answer= get_chat_response(prompt=prompt,
                              seed=seed,
                              model=model,
                              temperature=temperature  # The temperature parameter influences the randomness of the generated responses. A higher value, such as 0.8, makes the answers more diverse, while a lower value, like 0.2, makes them more focused and deterministic.
                              )

    answers_content.append((f'{TA[x]}.{loop_counter+1}', answer['response_content'])) # add the different replicats for answers over a single TA in a same list so i can analyse the similarity later
    answers_metadata.loc[x] = (f'{TA[x]}', #TA code
                               f'{loop_counter+1}', # replicate nbr
                               seed, 
                               temperature, 
                               answer["system_fingerprint"],
                               answer["prompt_tokens"],
                               answer["completion_tokens"]
                               ) 
    
    # Save response as csv file
    output_name = f'{date}output_{TA[x]}.{loop_counter+1}_s{seed}_t{temperature}.csv'

    with open((os.path.join(output_directory, output_name)), 'w') as f:
         f.write(answer["response_content"])

    # (Extra loop tools)
    loop_counter += 1         # incremental loop counter that resets to 0 every 3 loops so that it can add the ".1,2,3" at the end of each triplicats file names
    if loop_counter % 3 == 0:
        loop_counter = 0


# Save triplicats metadata as csv
answers_metadata.to_csv(path_or_buf= f'/Users/giorgiobolchi2/Documents/JRC/LLM/Data/Outputs/{date}/{date}output_s{seed}_t{temperature}_metadata.csv', 
                        sep=';', 
                        index=False)


KeyboardInterrupt: 

In [ ]:
# CODE TO CLEAN CSV FILES from first and last rows
output_directory = f'/Users/giorgiobolchi2/Documents/JRC/LLM/Data/Outputs/{date}'
output_directory = output_directory

for filename in os.listdir(output_directory):

    if 'TA' in filename and '.csv' in filename:     # Check if the file name contains the characters 'TA'
        
        # Open the file for reading and writing
        with open(f'{output_directory}{filename}', 'r+') as file:  # r+ = mode to open the file for both reading and writing
            lines = file.readlines()
            lines = lines[1:-1] # remove first and last row
            file.seek(0)  # move the file pointer to the beginning
            file.writelines(lines) # write the modified lines back to the file.
            file.truncate() # set the length of the file to the current position of the file pointer.


In [ ]:
## TEMP


seed = None 
temperature = 0.2
model = "gpt-4o"

date= '0131' 
output_directory = f'/Users/giorgiobolchi2/Documents/JRC/LLM/Data/Outputs/{date}/trial_1/'



In [11]:
# DATAFRAME COMPARISON SANDBOX


# Important note: you need to clean the generated csv files manually before running this following code block: 
#                 - if previous cleaning loop didn't work (it should work) -> remove the first line containing the characters ```csv and the last line with ```
#                 - it can happen that some columns (especially justifications and confidence_scores) are moved one cell to the right or left, move them back in the right place


date= date
seed= seed
temperature = temperature
output_directory = output_directory


#TA = ['TA1','TA2','TA3','TA4','TA5','TA6','TA7']
TA = ['TA5']

for x in range(len(TA)):

    # Define temporary dataframes for respective TA replicates 
    df1 = pd.read_csv(f'{output_directory}{date}output_{TA[x]}.1_s{seed}_t{temperature}.csv', sep=";")
    df2 = pd.read_csv(f'{output_directory}{date}output_{TA[x]}.2_s{seed}_t{temperature}.csv', sep=";")
    df3 = pd.read_csv(f'{output_directory}{date}output_{TA[x]}.3_s{seed}_t{temperature}.csv', sep=";")


    # Create a list of these DataFrames
    dfs = [df1, df2, df3]

    # Create a list to store the comparison tables
    comparisons = []

    # Compare replicats dataframes:
    for i in range(len(dfs)):
        for j in range(i+1, len(dfs)):
            comparison = dfs[i].reset_index(drop=True).eq(dfs[j].reset_index(drop=True)) # Reset the index before comparing (make it easier to compare)
            comparisons.append(comparison) # Add the comparison table to the list

    # Concatenate the comparison tables into a single DataFrame
    df_comparisons = pd.concat(comparisons, ignore_index=True)

    # Get information on the distribution of NA values per columns per replicat
    df_na = pd.DataFrame({f'{TA[x]}.1': df1.isna().sum(), 
                          f'{TA[x]}.2': df2.isna().sum(), 
                          f'{TA[x]}.3': df3.isna().sum()})
    
# Save results as csv
    output_directory = output_directory
    # Comparisons
    df_comparisons.to_csv(f'{output_directory}{date}comparisons_{TA[x]}.csv', sep=';', index=False)
    # NA distributions
    df_na.to_csv(f'{output_directory}{date}NAdistribution_{TA[x]}.csv', sep=';', index=True)

In [ ]:
test = df2['NACE_level1'].apply(lambda x: len(x.split()))

print(test)